# EEG Classification between normal and schizophrenic patients

In [33]:
#Preliminary Imports
from glob import glob
from scipy import stats

from tqdm import tqdm_notebook
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate,LeaveOneGroupOut

from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

## Loading the Dataset and Preprocessing EEG Data

In [2]:
#Loading the dataset
filepaths = glob('data/*.edf')
len(filepaths)

28

In [3]:
#Storing filepaths
healthy_filepaths = [filepath for filepath in filepaths if 'h' in filepath.split('\\')[1]]
patient_filepaths = [filepath for filepath in filepaths if 's' in filepath.split('\\')[1]]

In [4]:
#Function for reading EEG data from a single filepath
def read_data(filepath):
    data = mne.io.read_raw_edf(filepath, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    epochs = mne.make_fixed_length_epochs(data, duration=5, overlap=1)
    array = epochs.get_data()
    return array

In [5]:
#First healthy patient data
sample_data = read_data(healthy_filepaths[0])

Extracting EDF parameters from C:\Users\varma\Downloads\EEG Classification\data\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
231 matching events found
No baseline correction applied
0 projection ite

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


In [6]:
#(number of epochs, channels, length of signal)
sample_data.shape

(231, 19, 1250)

In [12]:
#Preprocessing the data into data_array, and storing the label_array and group_array
control_epoch_arrays = [read_data(i) for i in healthy_filepaths]
patient_epoch_arrays = [read_data(i) for i in patient_filepaths]

Extracting EDF parameters from C:\Users\varma\Downloads\EEG Classification\data\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
231 matching events found
No baseline correction applied
0 projection ite

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
227 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 227 events and 1250 original time points ...
0 bad epochs dropped
Extracting EDF parameters from C:\Users\varma\Downloads\EEG Classification\data\h03.edf...
EDF file detected
Setting c

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
227 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 227 events and 1250 original time points ...
0 bad epochs dropped
Extracting EDF parameters from C:\Users\varma\Downloads\EEG Classification\data\h04.edf...
EDF file detected
Setting c

...

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
283 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 283 events and 1250 original time points ...
0 bad epochs dropped
Extracting EDF parameters from C:\Users\varma\Downloads\EEG Classification\data\s14.edf...
EDF file detected
Setting c

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
542 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 542 events and 1250 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


0 bad epochs dropped


In [13]:
control_epoch_labels = [len(i)*[0] for i in control_epoch_arrays]
patient_epoch_labels = [len(i)*[1] for i in patient_epoch_arrays]

In [14]:
data_list = control_epoch_arrays + patient_epoch_arrays
label_list = control_epoch_labels + patient_epoch_labels

In [15]:
group_list = [[i]*len(j) for i,j in enumerate(data_list)]
len(group_list)

28

In [16]:
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(group_list)
print(data_array.shape, label_array.shape, group_array.shape)

(7201, 19, 1250) (7201,) (7201,)


## Feature Extraction

In [17]:
def mean(x):
    return np.mean(x,axis=-1)

In [18]:
def std(x):
    return np.std(x,axis=-1)

In [19]:
def ptp(x):
    return np.ptp(x,axis=-1)

In [20]:
def var(x):
    return np.var(x,axis=-1)

In [21]:
def minim(x):
    return np.min(x,axis=-1)

In [22]:
def maxim(x):
    return np.max(x,axis=-1)

In [23]:
def argminim(x):
    return np.argmin(x,axis=-1)

In [24]:
def argmaxim(x):
    return np.argmax(x,axis=-1)

In [25]:
def mean_square(data):
      return np.mean(data**2,axis=-1)

In [26]:
def rms(data):
      return np.sqrt(np.mean(data**2,axis=-1))

In [27]:
def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)

In [28]:
def skewness(data):
    return stats.skew(data,axis=-1)

In [29]:
def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

In [30]:
def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [34]:
features=[]
for data in tqdm_notebook(data_array):
    features.append(concatenate_features(data))
features=np.array(features)
features.shape

C:\Users\varma\AppData\Local\Temp\ipykernel_20108\1695981569.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data in tqdm_notebook(data_array):


  0%|          | 0/7201 [00:00<?, ?it/s]

(7201, 247)

## EEG Classification using Logistic Regression

In [35]:
clf=LogisticRegression()
gkf=GroupKFold(n_splits=5)
param_grid = {'classifier__C': [0.01,0.05,0.1,0.5, 1,2,3,4,5,8, 10,12,15]}
pipe=Pipeline([('scaler',StandardScaler()),('classifier',clf)])
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=16)
gscv.fit(features,label_array,groups=group_array)

C:\Users\varma\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', LogisticRegression())]),
             n_jobs=16,
             param_grid={'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5,
                                           8, 10, 12, 15]})

In [36]:
gscv.best_score_

0.6675662746091738

## EEG Classification using Convolutional Neural Networks (CNNs)

In [37]:
epochs_array = control_epoch_arrays + patient_epoch_arrays
epochs_array = np.vstack(epochs_array)

epochs_labels = control_epoch_labels + patient_epoch_labels
epochs_labels = np.hstack(epochs_labels)

group_array=np.hstack(group_list)

In [38]:
epochs_array.shape, epochs_labels.shape, group_array.shape

((7201, 19, 1250), (7201,), (7201,))

In [39]:
epochs_array=np.moveaxis(epochs_array,1,2)

In [40]:
epochs_array.shape

(7201, 1250, 19)

In [41]:
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(6250,19)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [42]:
model=cnnmodel()
model.summary()

C:\Users\varma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 6248, 5)             │             290 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 6248, 5)             │              20 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 6248, 5)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 3124, 5)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 3122, 5)             │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 3122, 5)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 1561, 5)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1561, 5)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 1559, 5)             │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 1559, 5)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling1d (AveragePooling1D) │ (None, 779, 5)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 779, 5)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 777, 5)              │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 777, 5)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling1d_1                  │ (None, 388, 5)              │               0 │
│ (AveragePooling1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 386, 5)              │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 386, 5)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 5)                   │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 636 (2.48 KB)

 Trainable params: 626 (2.45 KB)

 Non-trainable params: 10 (40.00 B)

In [46]:
gkf=GroupKFold()
accuracy=[]
for train_index, val_index in gkf.split(epochs_array, epochs_labels, groups=group_array):
    train_features,train_labels=epochs_array[train_index],epochs_labels[train_index]
    val_features,val_labels=epochs_array[val_index],epochs_labels[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=50,batch_size=128,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

C:\Users\varma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.5173 - loss: 0.6928 - val_accuracy: 0.3720 - val_loss: 0.7090
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5907 - loss: 0.6670 - val_accuracy: 0.3658 - val_loss: 0.7292
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.6073 - loss: 0.6438 - val_accuracy: 0.3686 - val_loss: 0.7473
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6932 - loss: 0.5759 - val_accuracy: 0.6809 - val_loss: 0.7284
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.8041 - loss: 0.4814 - val_accuracy: 0.6898 - val_loss: 0.6659
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.8136 - loss: 0.4646 - val_accuracy: 0.7303 - val_loss: 0.5793
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.8310 - loss: 0.4164 - val_accuracy: 0.7570 - val_loss: 0.5139
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.8669 - loss: 0.3859 - val_accuracy: 0.7612 - v

C:\Users\varma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.4799 - loss: 0.6970 - val_accuracy: 0.5376 - val_loss: 0.6860
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.5545 - loss: 0.6815 - val_accuracy: 0.5376 - val_loss: 0.6882
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.5656 - loss: 0.6429 - val_accuracy: 0.4076 - val_loss: 0.7587
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7411 - loss: 0.5659 - val_accuracy: 0.4110 - val_loss: 0.9802
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7979 - loss: 0.4656 - val_accuracy: 0.4841 - val_loss: 1.1736
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.8402 - loss: 0.3786 - val_accuracy: 0.5633 - val_loss: 1.2461
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.8666 - loss: 0.3308 - val_accuracy: 0.5443 - val_loss: 1.4366
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.8836 - loss: 0.3060 - val_accuracy: 0.5315 - v

C:\Users\varma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.5448 - loss: 0.6824 - val_accuracy: 0.6066 - val_loss: 0.6685
Epoch 2/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5225 - loss: 0.6630 - val_accuracy: 0.6066 - val_loss: 0.6666
Epoch 3/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6138 - loss: 0.6248 - val_accuracy: 0.6244 - val_loss: 0.6703
Epoch 4/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.7286 - loss: 0.5540 - val_accuracy: 0.6893 - val_loss: 0.6503
Epoch 5/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.7651 - loss: 0.4895 - val_accuracy: 0.7859 - val_loss: 0.6020
Epoch 6/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.8101 - loss: 0.4375 - val_accuracy: 0.7798 - val_loss: 0.6254
Epoch 7/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.8270 - loss: 0.4039 - val_accuracy: 0.7287 - val_loss: 0.7885
Epoch 8/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.8442 - loss: 0.3826 - val_accuracy: 0.7620 - v

C:\Users\varma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.5522 - loss: 0.6816 - val_accuracy: 0.5353 - val_loss: 0.6939
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.5626 - loss: 0.6479 - val_accuracy: 0.5972 - val_loss: 0.6808
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.6120 - loss: 0.6069 - val_accuracy: 0.5185 - val_loss: 0.6869
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.7451 - loss: 0.5422 - val_accuracy: 0.5003 - val_loss: 0.7206
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.8033 - loss: 0.4746 - val_accuracy: 0.5225 - val_loss: 0.7098
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8049 - loss: 0.4549 - val_accuracy: 0.5763 - val_loss: 0.6620
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.8313 - loss: 0.3997 - val_accuracy: 0.6597 - val_loss: 0.6515
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.8552 - loss: 0.3636 - val_accuracy: 0.6940 - v

C:\Users\varma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.5013 - loss: 0.6891 - val_accuracy: 0.6952 - val_loss: 0.6800
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.5231 - loss: 0.6656 - val_accuracy: 0.3479 - val_loss: 0.7156
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.6588 - loss: 0.6194 - val_accuracy: 0.3331 - val_loss: 0.8042
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.7484 - loss: 0.5451 - val_accuracy: 0.4482 - val_loss: 0.7398
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.7974 - loss: 0.4732 - val_accuracy: 0.6992 - val_loss: 0.6091
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.8161 - loss: 0.4226 - val_accuracy: 0.7954 - val_loss: 0.5129
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.8283 - loss: 0.4052 - val_accuracy: 0.7490 - val_loss: 0.5575
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.8389 - loss: 0.3892 - val_accuracy: 0.8022 - v

In [47]:
np.mean(accuracy)

0.6293159663677216